In [1]:
# import NumPy for numerical operations (arrays, reshaping, math, etc.)
import numpy as np

# import Pandas for data manipulation and analysis (DataFrames, CSV files, etc.)
import pandas as pd

In [2]:
# import function to split dataset into training and testing sets
from sklearn.model_selection import train_test_split

# import tool to apply different preprocessing steps to specific columns
from sklearn.compose import ColumnTransformer

# import imputer to handle missing values (e.g., fill with mean or most frequent)
from sklearn.impute import SimpleImputer

# import encoder to convert categorical variables into binary (0/1) format
from sklearn.preprocessing import OneHotEncoder

# import scaler to normalize numerical features between 0 and 1
from sklearn.preprocessing import MinMaxScaler

# import tools to build machine learning pipelines (combine multiple steps)
from sklearn.pipeline import Pipeline, make_pipeline

# import feature selection method to select top k features using Chi-squared test
from sklearn.feature_selection import SelectKBest, chi2

# import Decision Tree classifier model for classification tasks
from sklearn.tree import DecisionTreeClassifier

In [3]:
# load the CSV file named 'train.csv' from the 'data' folder into a DataFrame
df = pd.read_csv('train.csv')

# display the first 5 rows of the DataFrame to get a quick look at the data
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## **Pipeline Strategy**
1. Impute missing values (Age, Embarked)
2. OneHotEncoding (Sex, Embarked)
3. Scaling
4. Feature Selection (Top 5 columns only)
5. Model Training (DecisionTree)

In [4]:
# remove unnecessary columns from the dataset that won't help with prediction
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

# display the first 5 rows of the updated dataset
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## **Train test split**

In [5]:
# split the dataset into training and testing sets
# features: all columns except 'Survived'
# target: 'Survived' column (what we want to predict)
# 80% of the data is used for training, 20% for testing
# random_state ensures the same split every time (for reproducibility)
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=['Survived']),  # input features
    df['Survived'],                 # target variable
    test_size=0.2,                  # 20% for testing
    random_state=42                 # fixed seed for reproducible results
)

In [6]:
# display the first 5 rows of the training feature set
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [7]:
# display the first 5 values of the training labels (target variable)
y_train.head()

,Survived
331,0
733,0
382,0
704,0
813,0


## **1. Impute Transformer**
Note:
After using transformers like ColumnTransformer, the output becomes a NumPy array.
This means column names are lost after transformation.
So, in the next steps, we can't use column names — only column index numbers.
It's better to refer to columns by their index (e.g., [:, 0]) instead of names.

In [8]:
# create a ColumnTransformer to handle missing values
trf1 = ColumnTransformer([
    # fill missing values in column index 2 ('Age') using default strategy (mean)
    ('impute_age', SimpleImputer(), [2]),

    # fill missing values in column index 6 ('Embarked') using the most frequent value
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
],
    remainder='passthrough'  # keep all other columns as they are
)

## **2. One Hot Encoding Transformer**

In [9]:
# create a ColumnTransformer to apply OneHotEncoding on categorical columns
trf2 = ColumnTransformer([
    # apply OneHotEncoder to 'Sex' (index 1) and 'Embarked' (index 6)
    # - sparse_output=False: returns a dense NumPy array instead of sparse matrix
    # - handle_unknown='ignore': avoids errors for unseen categories in test data
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
],
    remainder='passthrough'  # keep all other columns unchanged
)

## **3. Scaling Transformer**

In [10]:
# create a ColumnTransformer to scale numeric features using MinMaxScaler
trf3 = ColumnTransformer([
    # apply MinMaxScaler to columns from index 0 to 9 (10 columns total)
    # this scales each value to a range between 0 and 1
    ('scale', MinMaxScaler(), slice(0, 10))
])

## **4. Feature Selection**

In [11]:
# select the top 8 best features based on the Chi-squared (chi2) statistical test
trf4 = SelectKBest(score_func=chi2, k=8)

## **5. Model**

In [12]:
# create a Decision Tree Classifier model for training and prediction
trf5 = DecisionTreeClassifier()

## **Create Pipeline**

In [13]:
# create a machine learning pipeline that chains all preprocessing and modeling steps
pipe = Pipeline([
    ('trf1', trf1),  # step 1: handle missing values (age & embarked)
    ('trf2', trf2),  # step 2: one-hot encode categorical features (sex & embarked)
    ('trf3', trf3),  # step 3: scale features between 0 and 1
    ('trf4', trf4),  # step 4: select top 8 important features using Chi-squared test
    ('trf5', trf5)   # step 5: train a Decision Tree Classifier
])

## **Pipeline VS make_pipeline**
Note:
Pipeline → we give names to each step &
make_pipeline → names are added automatically.
### **Same thing for ColumnTransformer VS make_column_transformer**

In [14]:
# show the full pipeline structure to check each step and what it does
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fdd1ccb4c20>)),
                ('trf5', DecisionTreeClassifier())])

In [15]:
# alternate syntax
# create a pipeline using make_pipeline (no need to name steps manually)
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [16]:
# display the structure of the pipeline and all its steps
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fdd1ccb4c20>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [17]:
# train (fit) the entire pipeline on the training data
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fdd1ccb4c20>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

## **Exploring Pipeline**

In [18]:
# view all steps in the pipeline with their auto-generated names (useful for accessing specific steps later)
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x7fdd1ccb4c20>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [19]:
# get the mean value computed by SimpleImputer for 'Age' (first transformer in columntransformer-1)
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [20]:
# get the most frequent value computed by SimpleImputer for 'Embarked' (second transformer in columntransformer-1)
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [21]:
# set scikit-learn to display a visual diagram of the pipeline instead of text
from sklearn import set_config
set_config(display='diagram')

In [22]:
# use the trained pipeline to make predictions on the test data
y_pred = pipe.predict(x_test)

In [23]:
# display the predicted output labels for the test data
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [24]:
# import function and calculate accuracy by comparing true and predicted labels
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464

## **Cross Validation Using Pipeline**

In [25]:
# perform 5-fold cross-validation to evaluate model accuracy more reliably
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)

## **GridSearch Using Pipeline (HyperParameter Tuning)**

In [26]:
# try different values for max_depth of the Decision Tree to find the best one
params = {
    'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, None]
}

In [27]:
# import GridSearchCV for hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# create a GridSearchCV object using the pipeline and parameter grid
# it will try each max_depth value using 5-fold cross-validation
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')

# fit the GridSearchCV on the training data to find the best max_depth
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unkno...ore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7fdd1ccb4c20>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [28]:
# show the best cross-validation accuracy score from GridSearchCV
grid.best_score_

np.float64(0.6391214419383433)

In [29]:
# show the best hyperparameter (max_depth) found by GridSearchCV
grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

## **Exporting the Pipeline**

In [30]:
# save the entire trained pipeline (including preprocessing and model) to a file
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))